In [7]:
%pip install -q ipycanvas
%pip install -q ipycc
import ipycanvas
import ipycc


In [8]:
from ipycc import Turtle


t = Turtle()
t.draw()
t.bgcolor('midnightblue')
t.pencolor('ghostwhite')
t.pendown()
side = 5
while side < t.width:
    t.forward(side)
    t.left(90)
    await t.delay(0.1)
    side += 5